In [ ]:
from experta import *
from recipes import recipes

class Ingredient(Fact):
    """Info about an ingredient."""
    pass

class MealType(Fact):
    """Info about the meal type (breakfast, lunch, dinner)."""
    pass

class NumberOfPeople(Fact):
    """Info about the number of people."""
    pass

class MealRecommender(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="find_meal")
    
    @Rule(Fact(action='find_meal'), NOT(MealType(type=W())))
    def ask_meal_type(self):
        valid_meal_types = ["breakfast", "lunch", "dinner"]
        while True:
            meal_type = input("What type of meal do you want? (breakfast/lunch/dinner): ").strip().lower()
            if meal_type in valid_meal_types:
                self.declare(MealType(type=meal_type))
                break
            else:
                print(f"Invalid input: '{meal_type}'. Please enter a valid meal type (breakfast/lunch/dinner).")
    
    @Rule(Fact(action='find_meal'), NOT(Ingredient(name=W())))
    def ask_ingredients(self):
        ingredients = input("Enter the ingredients you have, separated by commas: ").strip().lower().split(',')
        ingredients = [ingredient.strip() for ingredient in ingredients]
        confirmation = input(f"You entered the following ingredients: {', '.join(ingredients)}. Is this correct? (yes/no): ").strip().lower()
        if confirmation == 'yes':
            for ingredient in ingredients:
                self.declare(Ingredient(name=ingredient))
        else:
            print("Let's try again.")
            self.ask_ingredients()  # Retry input if confirmation fails
    
    @Rule(MealType(type=MATCH.type), AS.f << Fact(action='find_meal'))
    def find_meal(self, type, f):
        available_ingredients = {fact['name']: fact for fact in self.facts.values() if fact.__class__ == Ingredient}
        suggested_meals = []
        
        for recipe in recipes:
            if recipe['type'] == type:
                if isinstance(recipe['ingredients'], dict):
                    missing_ingredients = [ingredient for ingredient in recipe['ingredients'].keys() if ingredient not in available_ingredients]
                    if len(missing_ingredients) == 0:
                        suggested_meals.append(recipe)
                    elif len(missing_ingredients) == 1:
                        additional_ingredient = missing_ingredients[0]
                        response = input(f"You are missing '{additional_ingredient}' for '{recipe['name']}'. Can you get it? (yes/no): ").strip().lower()
                        if response == 'yes':
                            suggested_meals.append(recipe)
                else:
                    print(f"Error: Ingredients for {recipe['name']} are not in the expected format.")
        
        if suggested_meals:
            for meal in suggested_meals:
                servings = self.ask_number_of_people()
                if servings:
                    print(f"\nBased on the ingredients you have and your preference for {type}, you can make: ")
                    print(f"\n- {meal['name']}:")
                    print(f"  Ingredients (for {servings} people):")
                    for ingredient, quantity in meal['ingredients'].items():
                        adjusted_quantity = self.adjust_quantity(quantity, servings)
                        print(f"    {ingredient}: {adjusted_quantity}")
        else:
            print(f"\nSorry, we couldn't find any {type} recipes with the ingredients you have.")
        
        self.retract(f)
    
    def ask_number_of_people(self):
        while True:
            try:
                count = int(input("For how many people are you cooking?: ").strip())
                return count
            except ValueError:
                print("Invalid input. Please enter a valid number.")
    
    def adjust_quantity(self, quantity, servings):
        # Adjust the quantity based on the number of servings
        if " " in quantity:
            number, unit = quantity.split(" ", 1)
            try:
                total_quantity = float(number) * servings
                return f"{total_quantity} {unit}"
            except ValueError:
                return quantity  # In case quantity is not a number
        else:
            try:
                total_quantity = float(quantity) * servings
                return f"{total_quantity}"
            except ValueError:
                return quantity  # In case quantity is not a number

# Instantiate the expert system and run it
engine = MealRecommender()
engine.reset()  # Prepare the engine for the execution
engine.run()  # Run the engine


Enter the ingredients you have, separated by commas:  tortilla,turkey,lettuce,tomato,mayonnaise,bread,butter,cheddar cheese
You entered the following ingredients: tortilla, turkey, lettuce, tomato, mayonnaise, bread, butter, cheddar cheese. Is this correct? (yes/no):  yes
What type of meal do you want? (breakfast/lunch/dinner):  lunch


##### 